# MDAnalysis Notes

In [2]:
import MDAnalysis as mda
from MDAnalysis.analysis import *
from MDAnalysis.tests.datafiles import PSF, DCD, GRO, XTC
import warnings
import os

Universe properties and methods (holds master groups of atoms and topology objects)<br>
**Atoms:** All Atoms in the system (AtomGroup)<br>
**Residues:** All Residues in the system (ResidueGroup)<br>
**Segments:** All Segments in the system (chains?) (SegmentGroup)<br>
**Bonds:** All bond TopologyObjects in the systems<br>
**Angles:** All angle TopologyObjects in the systems<br>
**Dihedrals:** All dihedral TopologyObjects in the systems<br>
**Impropers:** All impropers TopologyObjects in the systems<br>

### Ens Monomer protein-ligand COM
Starting with a PDB file as input
Returns atom indices for SEEKR input files <br>
Active site residues defined as the protein backbone CA atoms within 5A of ligand <br>
Ligand atoms indices include all non-hydrogen heavy atoms <br>

In [3]:
#Define path, create universe
project_path = '/Users/javingfun/Library/CloudStorage/OneDrive-UCSanDiego/UCSD_PhD/Projects/Mpro_MD_dry'
pdb_path = '/Users/javingfun/Downloads/mpro_ens_monomer_solvated.pdb'
u_mpro = mda.Universe(pdb_path)

#Select atoms and make group
group = u_mpro.select_atoms('protein')
mpro_ca = u_mpro.select_atoms('protein and name CA')
mpro_ligand = u_mpro.select_atoms('resname 7YY and not name H*')

#Distance based selection
active_site = mda.analysis.distances.between(group,mpro_ca,mpro_ligand,5)
for residues in active_site.residues:
    print(residues)

#Create variables based on selection    
active_site_ca = active_site.select_atoms('name CA')
active_site_index = active_site_ca.atoms.indices
active_site_com = active_site_ca.center_of_mass()
ligand_index = mpro_ligand.indices

#Print variable output
print("Receptor CA atoms list: " + str(active_site_index))
print("Ligand atoms list: " + str((ligand_index)))
print('Center of Mass: ' + str(active_site_com))

<Residue THR, 24>
<Residue THR, 25>
<Residue THR, 26>
<Residue LEU, 27>
<Residue PRO, 39>
<Residue HIE, 41>
<Residue VAL, 42>
<Residue MET, 49>
<Residue TYR, 54>
<Residue PHE, 140>
<Residue LEU, 141>
<Residue ASN, 142>
<Residue GLY, 143>
<Residue SER, 144>
<Residue CYS, 145>
<Residue HIE, 163>
<Residue HID, 164>
<Residue MET, 165>
<Residue GLU, 166>
<Residue HIE, 172>
<Residue PHE, 181>
<Residue VAL, 186>
<Residue ASP, 187>
<Residue ARG, 188>
<Residue GLN, 189>
Receptor CA atoms list: [ 350  364  606 2164 2184 2203 2217 2224 2235 2509 2526 2543 2841 2853
 2877]
Ligand atoms list: [4682 4683 4684 4685 4686 4687 4688 4689 4690 4691 4692 4693 4694 4695
 4696 4697 4698 4699 4700 4701 4702 4703 4704 4705 4706 4707 4708 4709
 4710 4711 4712 4713 4714 4715 4716 4717 4718]
Center of Mass: [46.39793345 34.91706683 30.23126653]


In [4]:
#Write output to file
with open('mpro_ens_monomer_receptor_ligand_index.txt', 'w') as file:
    file.writelines('Receptor: ')
    file.writelines(str(active_site_index))
    file.writelines('\n Ligand: ')
    file.writelines(str(ligand_index))

### Apo Dimer active-site (COM) 
#### As defined by (Sztain, 2021)

In [5]:
#Go to OneDrive directory
os.chdir('/Users/javingfun/OneDrive - UC San Diego/UCSD_PhD/Projects/Mpro_MD_dry')

In [6]:
#Define path and import aligned structures (prepared with cpptraj)
topology = 'mpro_apo/mpro_apo_dimer/mpro_apo_dimer_dry.prmtop'
trajectory = 'mpro_apo/mpro_apo_dimer/mpro_apo_dimer_prod_aligned_dry_1.nc'

#Create universe
u = mda.Universe(topology,trajectory)

In [50]:
#Load universe as a PDB
pdb_path = '/Users/javingfun/Downloads/mpro_nat_asym_cluster_1_aligned.pdb'
u = mda.Universe(pdb_path)

In [49]:
#Combine all interface residues into one string
interface_a = [1,4,10,12,14,139,290,299,166,286]
interface_b = [x+306 for x in interface_a]
interface_all = []
interface_all.extend(interface_a)
interface_all.extend(interface_b)

interface = ''

for x in interface_all:
    interface += str(x) + ' '
    
active_a = [41,49,140,145,189,163,166,172]
active_b = [x+306 for x in active_a]

def make_atom_string(selection_list):
    selection = ''
    for x in selection_list:
        selection += str(x) + ' '
        
    return selection


In [51]:
#Select atoms

active_site_sztain = u.select_atoms('protein and resnum 7:198 and name CA')
active_site_a = u.select_atoms('protein and resnum 41 49 140 145 189 163 166 172 and name CA')
active_site_b = u.select_atoms('protein and resnum ' + make_atom_string(active_b) +' and name CA')

dimer = u.select_atoms('protein and name CA')
protomer_a = u.select_atoms('protein and resnum 1:306 and name CA')
protomer_b = u.select_atoms('protein and resnum 307:612 and name CA')
interface_com_sztain = mda.analysis.distances.between(dimer,protomer_a,protomer_b,3.5).center_of_mass()

interface_com = u.select_atoms('protein and resnum ' + interface + ' and name CA' ).center_of_mass()
                               
print('COM_Active_Sztain: ' + str(active_site_sztain.center_of_mass()))
print('COM_Active_A: ' + str(active_site_a.center_of_mass()))
print('COM_Active_B: ' + str(active_site_b.center_of_mass()))
print('\n')
print('COM_Interface_Sztain: ' + str(interface_com_sztain))
print('COM_Interface: ' + str(interface_com))

COM_Active_Sztain: [50.60176047 49.49027093 35.21789592]
COM_Active_A: [46.60150003 57.92275047 36.59762478]
COM_Active_B: [68.86887455 36.11100078 69.71249962]


COM_Interface_Sztain: [64.59249878 41.83100128 45.01049995]
COM_Interface: [56.38449974 48.06109982 54.68564949]


In [10]:
#function to get active site and center of mass for a given dimer
def get_com_dimer(u):
    
    active_site_a = u.select_atoms('protein and resnum 41 49 140 145 189 163 166 172 and name CA')
    active_site_b = u.select_atoms('protein and resnum ' + make_atom_string(active_b) +' and name CA')
    interface_com = u.select_atoms('protein and resnum ' + interface + ' and name CA' ).center_of_mass()
    
    print('COM_Active_Sztain: ' + str(active_site_sztain.center_of_mass()))
    print('COM_Active_A: ' + str(active_site_a.center_of_mass()))
    print('COM_Active_B: ' + str(active_site_b.center_of_mass()))
    print('\n')
    print('COM_Interface_Sztain: ' + str(interface_com_sztain))
    print('COM_Interface: ' + str(interface_com))

In [22]:
project_path = '/Users/javingfun/Library/CloudStorage/OneDrive-UCSanDiego/UCSD_PhD/Projects/Mpro_MD_dry'

apo_dimer_path = os.path.join(project_path,'mpro_apo/mpro_apo_dimer')
apo_dimer = mda.Universe(os.path.join(apo_dimer_path,'mpro_apo_dimer_dry.prmtop'),
                        os.path.join(apo_dimer_path,'mpro_apo_dimer_prod_aligned_dry_1.nc'))

get_com_dimer(apo_dimer)

COM_Active_Sztain: [50.46377927 49.77494337 35.26643966]
COM_Active_A: [46.17512703 57.96142006 36.88008881]
COM_Active_B: [68.33957195 35.3158021  69.17674208]


COM_Interface_Sztain: [64.86034584 41.83659744 45.34820938]
COM_Interface: [56.47282352 47.80187569 54.75079308]


In [23]:
nat_dimer_path = os.path.join(project_path,'mpro_nat/mpro_nat_dimer')
nat_dimer = mda.Universe(os.path.join(nat_dimer_path,'mpro_nat_dimer_dry.prmtop'),
                        os.path.join(nat_dimer_path,'mpro_nat_dimer_prod_aligned_dry_1.nc'))

get_com_dimer(nat_dimer)

COM_Active_Sztain: [50.46377927 49.77494337 35.26643966]
COM_Active_A: [35.94929123 40.20196462 38.86335993]
COM_Active_B: [63.41472006 75.49176025 43.14140081]


COM_Interface_Sztain: [64.86034584 41.83659744 45.34820938]
COM_Interface: [48.38204288 58.61683216 42.34641018]


In [29]:
nat_asym_path = os.path.join(project_path,'mpro_nat/mpro_nat_dimer_asym')
nat_asym = mda.Universe(os.path.join(nat_asym_path,'mpro_nat_dimer_asym_dry.prmtop'),
                        os.path.join(nat_asym_path,'mpro_nat_dimer_asym_prod_aligned_dry_1.nc'))

get_com_dimer(nat_dimer)

COM_Active_Sztain: [50.46377927 49.77494337 35.26643966]
COM_Active_A: [35.94929123 40.20196462 38.86335993]
COM_Active_B: [63.41472006 75.49176025 43.14140081]


COM_Interface_Sztain: [64.86034584 41.83659744 45.34820938]
COM_Interface: [48.38204288 58.61683216 42.34641018]


In [24]:
ens_dimer_path = os.path.join(project_path,'mpro_ens/mpro_ens_dimer')
ens_dimer = mda.Universe(os.path.join(ens_dimer_path,'mpro_ens_dimer_dry.prmtop'),
                        os.path.join(ens_dimer_path,'mpro_ens_dimer_prod_aligned_dry_1.nc'))

get_com_dimer(ens_dimer)

COM_Active_Sztain: [50.46377927 49.77494337 35.26643966]
COM_Active_A: [33.74974775 42.36769342 38.62882185]
COM_Active_B: [67.92099524 71.55962849 42.99357462]


COM_Interface_Sztain: [64.86034584 41.83659744 45.34820938]
COM_Interface: [49.87694206 58.47192936 42.27205563]


In [31]:
ens_asym_path = os.path.join(project_path,'mpro_ens/mpro_ens_dimer_asym')
ens_asym = mda.Universe(os.path.join(ens_asym_path,'mpro_ens_dimer_asym_dry.prmtop'),
                        os.path.join(ens_asym_path,'mpro_ens_dimer_asym_prod_aligned_dry_1.nc'))

get_com_dimer(ens_asym)

COM_Active_Sztain: [50.46377927 49.77494337 35.26643966]
COM_Active_A: [34.82932401 43.28239679 37.70007229]
COM_Active_B: [68.99399137 71.10424995 41.9344697 ]


COM_Interface_Sztain: [64.86034584 41.83659744 45.34820938]
COM_Interface: [50.86355743 57.73425255 41.41292362]


In [25]:
nirm_dimer_path = os.path.join(project_path,'mpro_nirm/mpro_nirm_dimer')
nirm_dimer = mda.Universe(os.path.join(nirm_dimer_path,'mpro_nirm_dimer_dry.prmtop'),
                        os.path.join(nirm_dimer_path,'mpro_nirm_dimer_prod_aligned_dry_1.nc'))

get_com_dimer(nirm_dimer)

COM_Active_Sztain: [50.46377927 49.77494337 35.26643966]
COM_Active_A: [47.81098448 45.40419551 30.73044014]
COM_Active_B: [24.43868501 60.71668243 66.32835334]


COM_Interface_Sztain: [64.86034584 41.83659744 45.34820938]
COM_Interface: [35.21558151 51.93758335 48.56791039]


In [30]:
nirm_asym_path = os.path.join(project_path,'mpro_nirm/mpro_nirm_dimer_asym')
nirm_asym = mda.Universe(os.path.join(nirm_asym_path,'mpro_nirm_dimer_asym_dry.prmtop'),
                        os.path.join(nirm_asym_path,'mpro_nirm_dimer_asym_prod_aligned_dry_1.nc'))

get_com_dimer(nirm_asym)

COM_Active_Sztain: [50.46377927 49.77494337 35.26643966]
COM_Active_A: [47.32420826 46.22183943 30.12676668]
COM_Active_B: [24.62404033 62.35144588 66.16711535]


COM_Interface_Sztain: [64.86034584 41.83659744 45.34820938]
COM_Interface: [34.65241499 52.4621767  48.26616764]
